**Pretraiment Part**

---

Nos donnees se presentent sous forme de trois elements:

-un fichier excel "Cases-Images-VIA" qui contient toutes les infromations sur les images telles que:le nom de l'image, le type d'image(avant application de l'acide acetique ou apres l'application)

-un fichier excel sur les meta-donnees de ces images a savoir le resultat du VIA "Positive", "Negative", "Suspicious of cancer" mais egalement d'autres donnees

-un fichier zippe qui contient les images 



Notre objectif est de regrouper dansun seul dataset les informations sur les photos, leur localisation et le resultat du VIA dans un meme fichier et de construire un modele de classification d'images capables de predire les resultats de l'examen

In [1]:
from google.colab import drive

import pandas as pd

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import os

In [5]:
os.chdir("/content/drive/MyDrive/Colab_Notebooks_data/col_uterus_datas/")

In [6]:
from posix import getcwd

In [ ]:
getcwd()

Lecture du fichier excel qui contient les informations sur les images

In [ ]:
images_via=pd.read_excel('Cases-Images-VIA',header=0)

In [ ]:
images_via

Lecture des meta-donnees relatives aux images

In [ ]:
labels_via=pd.read_excel('Cases-Meta-data-VIA',header=0)

In [ ]:
labels_via

Extraction dans les meta-donnees de la colonne qui concerne:


1.   Le cas auquel l'image fait partie(CaseID column)
2.   Le diagnistic de l'image(VIA column)




In [ ]:
labels=labels_via[['VIA','CaseID']]

Affichage des donnees

In [ ]:
labels

Labelisation de chaque CaseID

En effet dans cette partie on fait la chose suivante:
On sait que pour les images appartenant au meme cas en plus de leur nom ont les numeros 0,1 ou 2. Alors que pour les labels, ils ne concernent que le cas dans l'ensemble.


> Example: Pour le cas 118: nous avons au niveau des images ABY_0.jpg,ABY_1.jpg, ABY_2.jpg. Cependant au niveau des labels dans le deuxieme dataset, on a "ABY Positive". Donc on doit considerer doit transformer "ABY_1 Positive","ABY_2 Positive","ABY_0 Positive"



1. Ajouter les noms de tous les images dans le dataset des labels. Pour cela ajoutons pour chaque CaseID les numeros 0 et 1 

---

Ajout de 1

In [ ]:
for i in range(186):
  labels.loc[i+186]=labels.loc[i]
  line=labels.loc[i+186]
  line.CaseID=line.CaseID+"1"

Ajout de 0

In [ ]:
for i in range(186):
  labels.loc[i].CaseID=labels.loc[i].CaseID+"0"

In [ ]:
labels

En comptant le nombre d'enregistrements de chaque ligne on se rend compte d'un gap nentre lenombre d'images et le nombre de labels.
Une petite inspection du dataset nous montre que dans les images il y'en a qui ne concernent pas l'acide acetique 

In [ ]:
names=[]
for img in images_via['File']:
  name=img[:4]
  names.append(name)

In [ ]:
len(names)

Identification de ce gap entre les labels et les images

Verification d'images au Lugol

In [ ]:
indexes=[]
types=list(images_via['Type'])
#print(types)
for i in range(len(types)):
  if types[i]!="After application of acetic acid" and types[i]!="Before application of acetic acid":
    print(types[i])
    indexes.append(i)

In [ ]:
len(indexes)

0

Suppression de ces images relatives au Lugol

In [ ]:
images_via=images_via.drop(indexes)

In [ ]:
images_via

Dans cette partie, on rassemble les deux datasets: celui des images et celui des labels. On commence
 par creer une colonne en commun dans les deux datasets 

---


On renomme la colonne "CaseID" en "File"

In [ ]:
labels.rename(columns={'CaseID':'File'},inplace=True)

In [ ]:
labels

,VIA,File
0,Negative,AFC0
1,Negative,AJL0
2,Negative,AGY0
3,Negative,AJE0
4,Negative,AHS0
...,...,...
364,Suspicious of cancer,AAR1
365,Suspicious of cancer,AEY1
366,Suspicious of cancer,AEU1
367,Suspicious of cancer,ADF1


Merging des deux parties

In [ ]:
dataset_via=images_via.merge(labels)
dataset_via

**Commentaire**

---
On obtient a la fin un dataset avec deux colonnes: une colonne pour les noms des images et une autre qui contient les labels du test VIA

Supprimons CaseNumber 

In [ ]:
dataset_via=dataset_via.drop(columns='CaseNumber')

In [ ]:
dataset_via

**Recuperation des images**

---

Dezzipons le dossier des images

In [ ]:
!apt-get install unzip

In [ ]:
!unzip IARCImageBankVIA.zip

Rassemblons l'ensemble des images dans un nouveau dossier ImagesVia

In [ ]:
import glob
import shutil

In [ ]:
dst_dir = getcwd()+'/ImagesVia'
#print(dst_dir)
for i in range(1,187):
  src_dir = getcwd()+'/Case '
  src_dir=src_dir+str(i).zfill(3)
  #print(src_dir)
  for jpgfile in glob.iglob(os.path.join(src_dir, "*.jpg")):
    shutil.copy(jpgfile, dst_dir)

Ajout d'une nouvelle colonne qui contient les valeurs des chemins des images

In [ ]:
paths=[]
for i in range(368):
  paths.append(getcwd()+"/ImagesVia/"+dataset_via['File'][i])

In [ ]:
paths

In [ ]:
dataset_via['Paths']=paths

Enregistrement des donnees pretraitees dans un dataset

In [ ]:
dataset_via.to_csv("datas_via.csv")